In [10]:
import yfinance as yf
import pandas as pd
import glob
import os

In [119]:
### Load in option chain data ###

dataFolder = "/Users/sebastianbech/Desktop/Seminar - Asset prices and financial markets/Code/Data"

# List of all txt files in data folder
dataFiles = glob.glob(os.path.join(dataFolder, "*.txt")) 

# Columns to be used
columns = {' [QUOTE_DATE]'      : 'date',
           ' [UNDERLYING_LAST]' : 'underlyingLast', 
           ' [EXPIRE_DATE]'     : 'expiry',
           ' [DTE]'             : 'DTE',            # Days till Expiry
           ' [C_IV]'            : 'c_sigma',
           ' [C_LAST]'          : 'c_last',
           ' [STRIKE]'          : 'strike'
}

# Read all datafiles into separate dataframes
dataFiles_df = (pd.read_csv(file, 
                            low_memory=False, 
                            usecols = columns.keys(),
                            parse_dates = [' [QUOTE_DATE]']) for file in dataFiles)

# Concat dataframes for all data files into one dataframe
option_df   = (pd.concat(dataFiles_df, ignore_index=True)
               .rename(columns = columns)
               .convert_dtypes()
              )

# print shape of dataframe
print(f'Dataframe has {option_df.shape[0]:,} rows and {option_df.shape[1]} columns')


Dataframe has 3,834,774 rows and 7 columns


In [99]:
### Retrieve risk-free rate data ###


# de-annualize yearly interest rates
def deannualize(annual_rate, periods=365):
    return (1 + annual_rate) ** (1/periods) - 1

# download 3-month us treasury bills rates
annualized = yf.download("^IRX")["Adj Close"] / 100
    
# de-annualize
daily = annualized.apply(deannualize)

# create dataframe
rates = (pd.DataFrame({"annualized": annualized, "daily": daily})
         .reset_index()
         .rename(columns={'Date' : 'date'})
        )

# print shape of dataframe
print(f'Dataframe has {rates.shape[0]:,} rows and {rates.shape[1]} columns')

[*********************100%%**********************]  1 of 1 completed
Dataframe has 16,108 rows and 3 columns


/usr/local/anaconda3/lib/python3.10/site-packages/yfinance/utils.py:771: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')


In [147]:
translation = {
    'underlyingLast'  : 'S',
    'strike'           : 'K',
    'annualized'      : 'r',
    'DTE'             : 'T',
    'c_sigma'         : 'sigma',
    'c_last'          : 'observed_price'
}

# Join rates onto option chain data
option_prices = option_df.merge(rates, on='date', how='left')[translation.keys()].rename(columns=translation)

,S,K,r,T,sigma,observed_price
0,451.85,270.0,0.00040,0.0,,182.650000
1,451.85,275.0,0.00040,0.0,,0.000000
2,451.85,280.0,0.00040,0.0,,157.750000
3,451.85,285.0,0.00040,0.0,,153.450000
4,451.85,290.0,0.00040,0.0,,147.760000
...,...,...,...,...,...,...
3834769,380.36,570.0,0.00035,1022.0,0.186590,5.860000
3834770,380.36,575.0,0.00035,1022.0,0.184770,4.900000
3834771,380.36,580.0,0.00035,1022.0,0.187930,4.400000
3834772,380.36,585.0,0.00035,1022.0,0.184190,4.000000


,S,K,r,T,sigma,observed_price
0,451.85,270.0,0.00040,0.0,,182.650000
1,451.85,275.0,0.00040,0.0,,0.000000
2,451.85,280.0,0.00040,0.0,,157.750000
3,451.85,285.0,0.00040,0.0,,153.450000
4,451.85,290.0,0.00040,0.0,,147.760000
...,...,...,...,...,...,...
3834769,380.36,570.0,0.00035,1022.0,0.186590,5.860000
3834770,380.36,575.0,0.00035,1022.0,0.184770,4.900000
3834771,380.36,580.0,0.00035,1022.0,0.187930,4.400000
3834772,380.36,585.0,0.00035,1022.0,0.184190,4.000000


In [36]:
possibleColumns = [
    '[QUOTE_UNIXTIME]', 
    ' [QUOTE_READTIME]', 
    ' [QUOTE_DATE]',
    ' [QUOTE_TIME_HOURS]', 
    ' [UNDERLYING_LAST]', 
    ' [EXPIRE_DATE]',
    ' [EXPIRE_UNIX]', 
    ' [DTE]', 
    ' [C_DELTA]', 
    ' [C_GAMMA]', 
    ' [C_VEGA]',
    ' [C_THETA]', 
    ' [C_RHO]', 
    ' [C_IV]', 
    ' [C_VOLUME]', 
    ' [C_LAST]',
    ' [C_SIZE]', 
    ' [C_BID]', 
    ' [C_ASK]', 
    ' [STRIKE]', 
    ' [P_BID]',
    ' [P_ASK]', 
    ' [P_SIZE]',
    ' [P_LAST]', 
    ' [P_DELTA]', 
    ' [P_GAMMA]',
    ' [P_VEGA]', 
    ' [P_THETA]', 
    ' [P_RHO]', 
    ' [P_IV]', 
    ' [P_VOLUME]',
    ' [STRIKE_DISTANCE]', 
    ' [STRIKE_DISTANCE_PCT]'
]

"""
Field	Definition	Example
QUOTE_UNIXTIME	Time of quote as a Unix Timestamp	1583508900
QUOTE_READTIME	Time of quote in a readable form (year-month-day hour:minute) [yyyy-mm-dd hh:MM)	2020-03-06 10:35
QUOTE_DATE	Date of quote in a readable form (year-month-day) [yyyy-mm-dd]	2020-03-06
QUOTE_TIME_HOURS	Time of quoted day in hours	10.58333
UNDERLYING_LAST	Last price of underlying asset	296.38
EXPIRE_DATE	Expiry date of contract in a readable form (year-month-day) [yyyy-mm-dd]	2020-09-18
EXPIRE_UNIX	Expiry date of contract as a Unix Timestamp	1600459200
DTE	Days till Expiry in hours (assumes options expire on expiry date at 16:00)	196.18
C_DELTA	Derived Option Greek: Delta of Call option	0.12682
C_GAMMA	Derived Option Greek: Gamma of Call option	0.00508
C_VEGA	Derived Option Greek: Vega of Call option	0.45103
C_THETA	Derived Option Greek: Theta of Call option	-0.01843
C_RHO	Derived Option Greek: Rho of Call option	0.184
C_IV	Implied Volatility of Call option	0.18953
C_VOLUME	Days traded volume of Call option	1
C_LAST	Last traded price of Call option	2.25
C_SIZE	Current size of Call option	113 x 100
C_BID	Bid price of Call option	2.05
C_ASK	Ask price of Call option	2.82
STRIKE	Option strike price	345
P_BID	Bid price of Put option	52.58
P_ASK	Ask price of Put option	54.68
P_SIZE	Current size of Put option	113 x 113
P_LAST	Last traded price of Put option	43
P_DELTA	Derived Option Greek: Delta of Put option	-0.95081
P_GAMMA	Derived Option Greek: Gamma of Put option	0.00277
P_VEGA	Derived Option Greek: Vega of Put option	0.16209
P_THETA	Derived Option Greek: Theta of Put option	-0.02739
P_RHO	Derived Option Greek: Rho of Put option	-1.80885
P_IV	Implied Volatility of Put option	0.12222
P_VOLUME	Days traded volume of Put option	6
STRIKE_DISTANCE	Distance between strike price and current underlying price	48.6
STRIKE_DISTANCE_PCT	Distance as a percentage between strike price and current underlying price	0.164
"""

'\nField\tDefinition\tExample\nQUOTE_UNIXTIME\tTime of quote as a Unix Timestamp\t1583508900\nQUOTE_READTIME\tTime of quote in a readable form (year-month-day hour:minute) [yyyy-mm-dd hh:MM)\t2020-03-06 10:35\nQUOTE_DATE\tDate of quote in a readable form (year-month-day) [yyyy-mm-dd]\t2020-03-06\nQUOTE_TIME_HOURS\tTime of quoted day in hours\t10.58333\nUNDERLYING_LAST\tLast price of underlying asset\t296.38\nEXPIRE_DATE\tExpiry date of contract in a readable form (year-month-day) [yyyy-mm-dd]\t2020-09-18\nEXPIRE_UNIX\tExpiry date of contract as a Unix Timestamp\t1600459200\nDTE\tDays till Expiry in hours (assumes options expire on expiry date at 16:00)\t196.18\nC_DELTA\tDerived Option Greek: Delta of Call option\t0.12682\nC_GAMMA\tDerived Option Greek: Gamma of Call option\t0.00508\nC_VEGA\tDerived Option Greek: Vega of Call option\t0.45103\nC_THETA\tDerived Option Greek: Theta of Call option\t-0.01843\nC_RHO\tDerived Option Greek: Rho of Call option\t0.184\nC_IV\tImplied Volatility of

[*********************100%%**********************]  1 of 1 completed
            annualized     daily
Date                            
2024-05-13       5.240  0.005029
2024-05-14       5.235  0.005027
2024-05-15       5.240  0.005029
2024-05-16       5.243  0.005030
2024-05-17       5.240  0.005029


/usr/local/anaconda3/lib/python3.10/site-packages/yfinance/utils.py:771: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')


In [83]:
print(rates.head())

            annualized     daily
Date                            
1960-01-04        4.52  0.004691
1960-01-05        4.55  0.004706
1960-01-06        4.68  0.004770
1960-01-07        4.63  0.004746
1960-01-08        4.59  0.004726
